In [40]:
import geemap 
import ee 
import math
ee.Initialize()
#ee.Authenticate()


In [41]:
Map = geemap.Map()
roi01 = geemap.shp_to_ee(r"F:\随机森林\土壤湿度反演\region\area_gcs.shp").geometry()
#roi01 = ee.Geometry.Rectangle([113, 26, 119, 30], None, False)




In [42]:

Optical_dataset = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
SARdataset =ee.ImageCollection("COPERNICUS/S1_GRD")
dataset1 = Optical_dataset.filterBounds(roi01) \
                      .filterDate("2023-1-1","2023-12-31") \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))
image1=dataset1.median().clip(roi01).divide(10000)  #归一化处理 方便后续进行植被指数的计算
#Map.addLayer(image1, {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']},"RGB")

visParam = {
  'min': -1,
  'max': 1,
  'palette': ["FFFFFF", "CE7E45", "DF923D", "F1B555", "FCD163",
            "99B718", "74A901", "66A000", "529400", "3E8601",
            "207401", "056201", "004C00", "023B01", "012E01",
            "011D01", "011301"]
}
MNDWI01=image1.expression("(R-SWIR)/(R+SWIR)",{
                  "R":image1.select("B4"),"SWIR":image1.select("B11")}).rename('MNDWI')
#print(MNDWI01)
#Map.addLayer(MNDWI01, visParam, "MNDWI01")
# 筛选SAR影像并计算，找到一副与光学影像日期靠近的影像

#VV信号更强  IW模式分辨率高且范围广  两者为最佳搭配搭配

SARdataset = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(roi01) \
        .filterDate("2023-1-1","2023-12-31") \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) 
imgVV01 = SARdataset.median()


# 对用于水云模型的影像进行波段叠加
sar_vv= imgVV01.clip(roi01).select('VV').rename('all_ref')
sar_angle=imgVV01.clip(roi01).select('angle')
# m.addLayer(imgVV01.select('VV'), {min: -25, max: 5})
# m.addLayer(sar_vv, {'min': -25, 'max': 5})
# m.addLayer(sar_angle, {min: 0, max: 90})
all_layer = sar_vv.addBands(sar_angle).addBands(MNDWI01)


# 参照水云模型的公式对土壤后向散射系数求解
def soil_ref(image):
  # 计算植被层的双程衰减因子
  T2 = image.expression(
    'exp(-2 * 0.3334 * MNDWI / cos(angle))', {
      'MNDWI': image.select('MNDWI'),
      'angle': image.select('angle')})
  # 植被层后向反射系数
  veg_back = image.expression(
    '0.1449 * MNDWI * cos(angle) * (1 - T2)', {
      'MNDWI': image.select('MNDWI'),
      'angle': image.select('angle'),
      'T2': T2})
  # 计算土壤层后向反射系数
  soil_back = image.select('all_ref').subtract(veg_back).divide(T2)
  return soil_back
# 应用 soil_ref 函数
soil = soil_ref(all_layer)
#soil = soil.multiply(soil.gt(-50)).multiply(soil.lt(0))

#soil.getInfo() 
#geemap.download_ee_image(soil.clip(roi01), r"F:\随机森林\土壤湿度反演\img\soilk.tif", region=roi01, crs='epsg:4326', scale=30)

In [45]:
SM = ee.Image('projects/ee-qiumenghu88/assets/soil_moisture').rename('SM')
soil = soil.reproject(crs=SM.projection(), scale=10)
soil = soil.clip(roi01).rename('soil')
sample_points = SM.sample(region= roi01,scale= 10,numPixels=500,geometries= True)
#提取样本点特征
trainingData = soil.sampleRegions(collection=sample_points,
  properties=['SM'],  
  scale=10
)
#定义并训练分类器
rf = ee.Classifier.smileRandomForest(100)\
  .setOutputMode('REGRESSION')\
  .train(features=trainingData,
          classProperty='SM',
          inputProperties=['soil'])
classified = soil.classify(rf)
Map.addLayer(classified,{})
Map
#geemap.download_ee_image(soil_moisture.clip(roi01),r'F:\随机森林\土壤湿度反演\img\smmk.tif',region=roi01,scale=30,crs='EPSG:4527',max_tile_dim=1000)
#geemap.download_ee_image(classified.clip(roi01), r"F:\随机森林\土壤湿度反演\img\smm.tif",region=roi01,scale=30,crs='EPSG:4527',max_tile_dim=1000)
#geemap.download_ee_image(soil.clip(roi01), r"F:\随机森林\土壤湿度反演\img\soilk.tif", region=roi01, crs='epsg:4326', scale=30,max_tile_dim=1000)
geemap.ee_export_image_to_drive(classified.clip(roi01),description = 'soil_moisture',region=roi01,scale = 10,maxPixels = 1e13)